# Load group data
Here we use the group information including descriptions and rules as an example for how to do basic processing on our text data. Similar strategy can be used in processing texts in group admin profile, admin tags, etc.

In [ ]:
import pandas as pd
import numpy as np
groups = pd.read_csv("Nature_Groups.csv", usecols = [0])

In [ ]:
group_id = groups['groupID'].tolist()

# Get group info(i.e. description/rules) via API call

In [ ]:
import oauth2
import time
import urllib.request
import pandas as pd

#Define function to call data through API
def build_request(url, method='GET', groupid = None):
    params = {                                            
        'oauth_version': "1.0",
        'oauth_nonce': oauth2.generate_nonce(),
        'oauth_timestamp': int(time.time()),
        'method': 'flickr.groups.getInfo',
        'group_id': groupid,
        'format': 'json',
        'nojsoncallback':'1'     
    }
    consumer = oauth2.Consumer(key='cc6e0093965331473ef6d9be604b5a96', secret='a758040164b10f62')
    token = oauth2.Token(key='72157709455713622-99a678a48136490f', secret='4a13c4e35c898f21')

    params['oauth_consumer_key'] = consumer.key
    params['oauth_token']= token.key


    req = oauth2.Request(method=method, url=url, parameters=params)
    signature_method = oauth2.SignatureMethod_HMAC_SHA1()
    req.sign_request(signature_method, consumer, token)
    return req

In [ ]:
import json as js
groupinfo = []      
for i in group_id:     
    print(i)                
    each_group = []       
    request = build_request('https://www.flickr.com/services/rest', groupid = str(i))
    u = urllib.request.urlopen(request.to_url())
    u = js.loads(u.read())
    each_group.append(u)
    groupinfo.append(each_group)    

## Extract description and rule respectively

In [ ]:
#Look up description and rule for each group
from nested_lookup import nested_lookup
group_descr = {}
group_rule = {}
for i in range(len(groupinfo)):
    j = groupinfo[i]
    k = nested_lookup('description', j)
    l = nested_lookup('rule', j)
    group_descr[group_id[i]] = k[i]
    group_rule[group_id[i]] = l[i]

#Ourput as JSON
description = list(group_descr.items())
rule = list(group_rule.items())
for u in range(len(descriptipn)):
        t = description[u]
        s = rule[u]
        u_str = str(u+1)
        d_file_name = 'DESCR_GR' + u_str + '_' + str(t[0])+ '.json'
        r_file_name = 'RULE_GR' + u_str + '_' + str(t[0])+ '.json'
        d = open("flickr/group_info/description" + d_file_name,'w')
        r = open("flickr/group_info/rule" + d_file_name,'w')
        js.dump(t, d)
        js.dump(s, r)
        d.close()
        r.close()

# Basic text processing
## Group description

In [ ]:
#Text Processing
#Clean texts
from cleantext import clean
descr = list(group_descr.values())
clean_descr = []
for i in range(len(descr)):
    each_group = []
    e_descr = descr[i]
    clean_text = clean(e_descr,
        fix_unicode = True,               # fix various unicode errors
        to_ascii = True,                  # transliterate to closest ASCII representation
        lower = True,                     # lowercase text
        no_line_breaks = True,           # fully strip line breaks as opposed to only normalizing them
        no_urls = True,                  # replace all URLs with a special token
        no_emails = True,                # replace all email addresses with a special token
        no_phone_numbers = True,         # replace all phone numbers with a special token
        no_numbers = False,               # replace all numbers with a special token
        no_digits = False,                # replace all digits with a special token
        no_currency_symbols = True,      # replace all currency symbols with a special token
        no_punct = True,                 # fully remove punctuation
    #replace_with_url="<URL>",
        replace_with_email = "<EMAIL>",
        replace_with_phone_number = "<PHONE>",
        replace_with_number = "<NUMBER>",
        replace_with_digit = "0",
        replace_with_currency_symbol = "<CUR>",
        lang = "en")                    
    each_group.append(clean_text)
    clean_descr.append(each_group)

In [ ]:
# Further processing
# Remove stopwords
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from nltk.tokenize import word_tokenize

nostopwords_descr = []
for i in range(len(clean_descr)):
    each = clean_descr[i]
    eachgroup = []
    for j in range(len(each)):
        k = each[j]
        tokens = word_tokenize(k)
        remove_stopwords = [s for s in tokens if not s in stop_words]
        eachgroup.append(remove_stopwords)
    nostopwords_descr.append(eachgroup)

In [ ]:
#Sort words in terms of times of occurrence
from collections import Counter
sorted_descrwords = []
for i in range(len(subject)):
    j = subject[i]
    m = Counter(item for item in j)
    l = dict(m)
    l_sorted = {k: v for k, v in sorted(l.items(), key=lambda x: x[1], reverse = True)}
    sorted_descrwords.append(l_sorted)

sorted_descr = {}
for i in range(len(group_id):
    sorted_descr[group_id[i]] = list(sorted_descrwords.keys())[i]

df_descrwords = pd.DataFrame.from_dict(sorted_descr, orient='index')
df_descrwords = df_descrwords.fillna('')
df_descrwords = df_descrwords.transpose()
df_descrwords.to_csv('Group_Descr_Words.csv')

#Similar for processing group rules

# Calculate cosine similarity across groups after word embedding using pre-trained GloVe model
Here we take the group description as an example, similar strategy can be utilized in the group rules and admin tag, etc.

## Load pre-trained GloVe model

In [ ]:
import numpy as np
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding="utf8")
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

model = loadGloveModel(r"C:\Users\lunad\Downloads\glove.6B\glove.6B.50d.txt")

## Define function to calculate cosine similarity

In [ ]:
def cosine_distance_wordembedding_method(s1, s2):
    import scipy
    vector_1 = np.mean([model[word] for word in s1],axis=0)
    vector_2 = np.mean([model[word] for word in s2],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    percent = str(round((1-cosine)*100,2)) + "%"
    return percent

## Create a list od group descriptions for calculation

In [ ]:
descr_words = group_descr.values.tolist()
group_id = group_descr.keys.tolist()

## Loop through each pair of groups to calculate their cosine similarity on description

In [ ]:
cos_sim = []
for a in descr_words:
    e_d = []
    for b in descr_words:
        c = cosine_distance_wordembedding_method(a,b)
        e_d.append(c)
    cos_sim.append(e_r)

In [ ]:
cos_similarity_descr = pd.DataFrame(cos_sim, columns = group_id, index = group_id)